In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import numpy as np
from functools import partial

In [ ]:
import sys, os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))

In [ ]:
from maxvol_compression.sketch_matrix import FastFrequentDirections, RandomSums
from utils.dummy import DummyDatasetCifar10, DummyModelCifar10

In [ ]:
BATCH_SIZE = 10

In [ ]:
model = DummyModelCifar10()
cifar10 = DummyDatasetCifar10(batch_size=BATCH_SIZE, data_root='../data')

In [ ]:
model.eval()

In [ ]:
# Test run to compute bounds
fds = FastFrequentDirections(150, 400, keep_original=True)
def update_sketchmatrix(self, input, output, alg):
    batch_size = input[0].shape[0]
    alg.update(input[0].view(batch_size, -1).cpu().numpy())
    
handle = model.fc1.register_forward_hook(
    partial(update_sketchmatrix, alg=fds))

In [ ]:
# handle.remove()

In [ ]:
%%time
for i, (batch,_) in enumerate(cifar10.testloader, 1):
    with torch.no_grad():
        _ = model(batch)
    if i % 200 == 0: print(f'{i} batches completed')
handle.remove()

In [ ]:
fds.sketch_matrix[:10]

In [ ]:
fds.compute_error()

In [ ]:
# Test run to compute bounds
rs = RandomSums(150, 400, keep_original=True)
def update_sketchmatrix(self, input, output, alg):
    batch_size = input[0].shape[0]
    alg.update(input[0].view(batch_size, -1).cpu().numpy())
    
handle = model.fc1.register_forward_hook(
    partial(update_sketchmatrix, alg=rs))

In [ ]:
%%time
for i, (batch,_) in enumerate(cifar10.testloader, 1):
    with torch.no_grad():
        _ = model(batch)
    if i % 200 == 0: print(f'{i} batches completed')
handle.remove()

In [ ]:
rs.sketch_matrix

In [ ]:
rs.compute_error()